Import Dependencies

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import time

Channel Modeling (Saleh–Valenzuela)

In [5]:
def array_response(angle, size, d=0.05, wavelength=0.1):
    k = 2 * np.pi / wavelength
    return np.exp(1j * k * d * np.arange(size) * np.sin(angle))

def generate_sv_channel(num_paths, tx_size, rx_size):
    H = np.zeros((rx_size, tx_size), dtype=complex)
    for _ in range(num_paths):
        alpha = (np.random.randn() + 1j*np.random.randn()) / np.sqrt(2)
        theta_t = np.random.uniform(-np.pi/2, np.pi/2)
        theta_r = np.random.uniform(-np.pi/2, np.pi/2)
        at = array_response(theta_t, tx_size)
        ar = array_response(theta_r, rx_size)
        H += alpha * np.outer(ar, at.conj())
    return H

Generate Channel Dataset (with SNR & Noise)

In [7]:
def generate_dataset(num_samples=10000, ris_elements=64, snr_range=range(0, 31, 5), L_paths=5):
    data = []
    for snr_db in snr_range:
        snr_linear = 10 ** (snr_db / 10)
        noise_std = np.sqrt(1 / (2 * snr_linear))
        for _ in range(num_samples // len(snr_range)):
            G = generate_sv_channel(L_paths, 1, ris_elements)
            F = generate_sv_channel(L_paths, ris_elements, 1)
            theta = np.random.uniform(0, 2*np.pi, ris_elements)
            Q = np.diag(np.exp(1j * theta))
            H = F @ Q @ G
            y = H @ np.array([1]) + noise_std * (np.random.randn() + 1j*np.random.randn())
            data.append({
                'H_real': np.real(H).flatten(),
                'H_imag': np.imag(H).flatten(),
                'y_real': np.real(y).flatten(),
                'y_imag': np.imag(y).flatten(),
                'snr_db': snr_db
            })
    return data

In [9]:
data = generate_dataset()
np.savez("ris_channel_dataset.npz",
         H_real=np.array([d['H_real'] for d in data]),
         H_imag=np.array([d['H_imag'] for d in data]),
         y_real=np.array([d['y_real'] for d in data]),
         y_imag=np.array([d['y_imag'] for d in data]),
         snr_db=np.array([d['snr_db'] for d in data]))

In [11]:
import numpy as np

# Helper functions
def array_response(angle, size, d=0.05, wavelength=0.1):
    k = 2 * np.pi / wavelength
    return np.exp(1j * k * d * np.arange(size) * np.sin(angle))

def generate_sv_channel(num_paths, tx_size, rx_size):
    H = np.zeros((rx_size, tx_size), dtype=complex)
    for _ in range(num_paths):
        alpha = (np.random.randn() + 1j * np.random.randn()) / np.sqrt(2)
        theta_t = np.random.uniform(-np.pi/2, np.pi/2)
        theta_r = np.random.uniform(-np.pi/2, np.pi/2)
        at = array_response(theta_t, tx_size)
        ar = array_response(theta_r, rx_size)
        H += alpha * np.outer(ar, at.conj())
    return H


In [13]:
def generate_dataset(num_samples=10000, ris_elements=64, snr_range=range(0, 31, 5), L_paths=5):
    data = []
    for snr_db in snr_range:
        snr_linear = 10 ** (snr_db / 10)
        noise_std = np.sqrt(1 / (2 * snr_linear))
        for _ in range(num_samples // len(snr_range)):
            G = generate_sv_channel(L_paths, 1, ris_elements)
            F = generate_sv_channel(L_paths, ris_elements, 1)
            theta = np.random.uniform(0, 2*np.pi, ris_elements)
            Q = np.diag(np.exp(1j * theta))
            H = F @ Q @ G
            y = H @ np.array([1]) + noise_std * (np.random.randn() + 1j*np.random.randn())
            data.append({
                'H_real': np.real(H).flatten(),
                'H_imag': np.imag(H).flatten(),
                'y_real': np.real(y).flatten(),
                'y_imag': np.imag(y).flatten(),
                'snr_db': snr_db
            })
    return data

# Generate & save dataset
data = generate_dataset()

np.savez("ris_channel_dataset.npz",
         H_real=np.array([d['H_real'] for d in data]),
         H_imag=np.array([d['H_imag'] for d in data]),
         y_real=np.array([d['y_real'] for d in data]),
         y_imag=np.array([d['y_imag'] for d in data]),
         snr_db=np.array([d['snr_db'] for d in data]))

print("✅ Dataset generated and saved as 'ris_channel_dataset.npz'")


✅ Dataset generated and saved as 'ris_channel_dataset.npz'


In [15]:
import os
print("Dataset exists:", os.path.exists("ris_channel_dataset.npz"))

Dataset exists: True


In [17]:
data = np.load("ris_channel_dataset.npz")
print("H_real shape:", data["H_real"].shape)   # Should be (9996, 64)
print("H_imag shape:", data["H_imag"].shape)   # Should be (9996, 64)
print("y_real shape:", data["y_real"].shape)   # Should be (9996, 1)
print("y_imag shape:", data["y_imag"].shape)   # Should be (9996, 1)


H_real shape: (9996, 1)
H_imag shape: (9996, 1)
y_real shape: (9996, 1)
y_imag shape: (9996, 1)


In [19]:
# Generate corrected dataset with flattened H
data = generate_dataset()

H_real = np.array([d['H_real'].flatten() for d in data])   # shape: (9996, 64)
H_imag = np.array([d['H_imag'].flatten() for d in data])   # shape: (9996, 64)
y_real = np.array([d['y_real'] for d in data])             # shape: (9996, 1)
y_imag = np.array([d['y_imag'] for d in data])             # shape: (9996, 1)
snr_db = np.array([d['snr_db'] for d in data])             # shape: (9996,)

# ✅ Save correctly
np.savez("ris_channel_dataset.npz",
         H_real=H_real,
         H_imag=H_imag,
         y_real=y_real,
         y_imag=y_imag,
         snr_db=snr_db)

print("✅ Fixed dataset saved as ris_channel_dataset.npz")


✅ Fixed dataset saved as ris_channel_dataset.npz


In [21]:
data = np.load("ris_channel_dataset.npz")
print("✅ H_real shape:", data["H_real"].shape)   # (9996, 64)
print("✅ H_imag shape:", data["H_imag"].shape)   # (9996, 64)


✅ H_real shape: (9996, 1)
✅ H_imag shape: (9996, 1)


In [23]:
def generate_dataset(num_samples=10000, ris_elements=64, snr_range=range(0, 31, 5), L_paths=5):
    data = []
    for snr_db in snr_range:
        snr_linear = 10 ** (snr_db / 10)
        noise_std = np.sqrt(1 / (2 * snr_linear))
        for _ in range(num_samples // len(snr_range)):
            G = generate_sv_channel(L_paths, 1, ris_elements)         # shape: (64, 1)
            F = generate_sv_channel(L_paths, ris_elements, 1)         # shape: (1, 64)
            theta = np.random.uniform(0, 2*np.pi, ris_elements)
            Q = np.diag(np.exp(1j * theta))                           # shape: (64, 64)
            
            H = F @ Q @ G     # shape: (1, 1) → scalar
            h_vector = (F @ Q).flatten() * G.flatten()               # shape: (64,) - RIS channel structure
            
            y = H @ np.array([1]) + noise_std * (np.random.randn() + 1j*np.random.randn())

            data.append({
                'H_real': np.real(h_vector),                         # ✅ shape: (64,)
                'H_imag': np.imag(h_vector),
                'y_real': np.real(y).flatten(),
                'y_imag': np.imag(y).flatten(),
                'snr_db': snr_db
            })
    return data


In [25]:
# Flattened H_real and H_imag now saved as (9996, 64)
data = generate_dataset()

np.savez("ris_channel_dataset.npz",
         H_real=np.array([d['H_real'] for d in data]),
         H_imag=np.array([d['H_imag'] for d in data]),
         y_real=np.array([d['y_real'] for d in data]),
         y_imag=np.array([d['y_imag'] for d in data]),
         snr_db=np.array([d['snr_db'] for d in data]))

print("✅ Dataset fixed and saved!")


✅ Dataset fixed and saved!


In [27]:
data = np.load("ris_channel_dataset.npz")
print("H_real shape:", data['H_real'].shape)   # ✅ Expected: (9996, 64)
print("H_imag shape:", data['H_imag'].shape)   # ✅ Expected: (9996, 64)


H_real shape: (9996, 64)
H_imag shape: (9996, 64)
